#  Leetcodeの復習

あえて言語化にこだわる。
とりあえず、新しく解いた問題を上に書き足していくが、もし関連する過去に解いた問があれば、それを浮上させてくる。


## 227. [Basic Calculator II](https://leetcode.com/problems/basic-calculator-ii/)

### Sol1. traverse with stack

### Sol2. traversion without stack

### Sol3. Iterator with stack

### Sol4. Iterator with stack


```py

class Solution:
    def calculate(self, s: str) -> int:
        itr = StringIterator(s)
        ret = 0
        last_operation = None
        
        while itr.has_next():
            is_muliplication_or_division, curr_ope = itr.next()
            if is_muliplication_or_division:
                last_operation = lambda x, last_operation=last_operation, curr_ope=curr_ope: x+curr_ope(last_operation(0))
            else:
                if last_operation is not None:
                    ret += last_operation(0)
                last_operation = curr_ope
        return ret + last_operation(0)
  
        
class StringIterator:
    def __init__(self, string):
        if string[0] != '-':
            self.string = ('+' + string).replace(' ', '')
        self.curr_idx = -1
        
        
    def has_next(self):
        return self.curr_idx + 1 <= len(self.string) - 1
    
    def next(self):
        self.curr_idx += 1
        operation_str = self.string[self.curr_idx]
        temp = self.curr_idx + 1
        while (self.curr_idx + 1 < len(self.string)) and self.string[self.curr_idx + 1].isdigit():
            self.curr_idx += 1
        num = int(self.string[temp:self.curr_idx+1])

        operation_to_lambda = {
            '+': lambda x, num=num: x + num,
            '-': lambda x, num=num: x - num,
            '*': lambda x, num=num: x * num,
            '/': lambda x, num=num: x // num if x > 0 else -((-x)//num),  # ここのif節最初考えてなかった
        }
        
        return (operation_str in ['*', '/']), operation_to_lambda[operand_str]
```

### 所感

自力でsol3を思いついた。その後、sol4にk


- Iteratorを使ったら綺麗かなーと思って使ったところ、iteratorに処理を押し込めれたのは確かに綺麗だが、代わりにラムダ関数が発生してそこが辛くなった。ラムダ関数の挙動をちゃんと理解していなくて、30分くらい詰まった。https://twitter.com/kdjp20/status/1426855305032339460
- `operation_to_lambda['/']` の負の場合の条件分岐を入れていなかった。
- operation(演算子+,-,,)をoperand(被演算子)と命名していたw

## [146. LRU Cache](https://leetcode.com/problems/lru-cache/)



### 所感

220の解法2を思い出すついでにsortedset, orderedset周りに関してまとめた。 https://twitter.com/kdjp20/status/1426808248246497282 ので、ordered set繋がりで復習する。

言わずもがなの超重要問題。

## 220. [Contains Duplicate III]()


### 解法2 SortedSet O(Nlog(min(N, k)))time

```py
    def containsNearbyAlmostDuplicate(self, nums: List[int], k: int, t: int) -> bool:
        nums_in_window = SortedSet()
        
        for right in range(len(nums)):
            if k < right:
                nums_in_window.remove(nums[right - k - 1])    # logk
            idx = bisect.bisect_left(nums_in_window, nums[right] - t)     # logk
            if idx <= len(nums_in_window)-1 and nums_in_window[idx] <= nums[right] + t:
                return True
            nums_in_window.add(nums[right])     # logk
            
        return False
```

pythonのstdlibにはsorted setはないので、3rd party libのsortedcontainers.SortedSetを使うか、自前でBalanced BSTを使用してsorted setを実装する必要がある(しんどい)。

### 解法3 Buckets O(N) time


```py
    def containsNearbyAlmostDuplicate(self, nums: List[int], k: int, t: int) -> bool:
        # id of the bucket is defined by x // t+1
        # key is id and value is number which is in bucket.
        # Note that multiple numbers cannot be in the same bucket, because in that case,
        # True should have been returned.
        buckets = dict()
        for right in range(len(nums)):
            if k < right:
                del buckets[self.get_bucket_id(t+1, nums[right - k - 1])]
            
            bucket_id = self.get_bucket_id(t+1, nums[right])
            if bucket_id in buckets:
                return True
            if (bucket_id - 1 in buckets) and abs(buckets[bucket_id - 1] - nums[right]) <= t:
                return True
            if (bucket_id + 1 in buckets) and abs(buckets[bucket_id + 1] - nums[right]) <= t:
                return True
            
            buckets[bucket_id] = nums[right]
        return False
            
    def get_bucket_id(self, bucket_size, num):
        return num // bucket_size
```






バケツの大きさはt+1が良い理由は、See: https://twitter.com/kdjp20/status/1426827866558189569





### 所感


この問題、実は五ヶ月前の21/03/14にすでに解いていたが、完全に忘れていた。   
解法2を思い出すついでにsortedset, orderedset周りに関してまとめた。 https://twitter.com/kdjp20/status/1426808248246497282    
解法3天才。[←村人さんのおすすめ](https://twitter.com/ronin_2020/status/1371056641903194113)    

解法2も解法3も、「同一windowには重複する要素は入っていない、 同一バケツ内には複数の要素は入っていない。なぜならもしそんなことが起こっていたらそれらの要素が入る時点ですでにTrueが返されているはずだから」と言うのが面白いし天才的なところ。



## 219. [Contains Duplicate II](https://leetcode.com/problems/contains-duplicate-ii/)

### 所感

~~簡単。~~良問すぎる。以下3つも学んだ。


- space complexiyは、O(min(n, k)) assuming that n = length of a given list。
   - 確かに。脱帽。
- 最初のk個とそれ以降のk個のそれぞれのチェックに関しては、以下のような実装により、一つのループにまとめられる。
   - 確かに。それはそうだ。


```py
class Solution:
    # なお、k=0の場合もちゃんとバグらずに動いてくれる
    def containsNearbyDuplicate(self, nums: List[int], k: int) -> bool:
        left = 0
        right = 0
        num_to_cnt = defaultdict(int)
        num_to_cnt[nums[0]] = 1
        
        while right + 1 < len(nums):
            if right - left >= k:      ### このif節が肝。
                num_to_cnt[nums[left]] -= 1
                left += 1
            
            if num_to_cnt[nums[right + 1]] > 0:
                return True
            right += 1
            num_to_cnt[nums[right]] += 1
        return False
```

- しかも、この問題、実は要素のcntをキープしなくても良い！！！
    - 例えばleft+=1で除外すべき数字が、slideing windowの範囲内に他にも存在するかもしれないので安易に `del num_set[left]`してはいけないと思っていた。が、もし、他の場所に存在するのならば、その数字にrightが到達した時点でreturn Trueされているはずだったので、ないと考えて良い。天才。
    - とするともはやleftとrightを追う必要もない。
    
```py

class Solution:
    def containsNearbyDuplicate(self, nums: List[int], k: int) -> bool:
        nums_set = set()
        
        for right in range(len(nums)):
            if k < right:
                nums_set.remove(nums[right-k-1])
            if nums[right] in nums_set:
                return True
            nums_set.add(nums[right])
        return False
```

## 1175. [Prime Arrangements](https://leetcode.com/problems/prime-arrangements/)




### 所感

素数判定&素数カウントをちゃんとエラトステネスの篩で実装したのは地味に初めてかもしれない。    
1~Nまでの素数(の数)を求めるには、


```python
        # 最初のFalseは、listのindexとiを合わせるためのpseudo element
        are_primes = [False, False] + [True] * (n-1)
        for num in range(2, math.ceil(n ** 0.5)+1):    # +1 するの忘れてた
            if not are_primes[num]:
                continue
            i = 2
            while num * i <= n:
                are_primes[num * i] = False
                i += 1
# n = 5 の場合、are_primes = [False, False, True, True, False, True]となる。

```

## 307. [Range Sum Query - Mutable](https://leetcode.com/problems/range-sum-query-mutable/)



### 所感

もはやcumulative sum関係ないが、ついでにこれも復習する。これは、cumulative sumを使うと、更新時にO(N)かかってしまうのでだめ。

## [303. Range Sum Query - Immutable](https://leetcode.com/problems/range-sum-query-immutable/)


### 所感

queryがたくさん投げられるのでcumulative sumのリストを作っておくという典型問題だった。

## 53. [Maximum Subarray](https://leetcode.com/problems/maximum-subarray/)



###  cumulative sum  = O(n**2) time


よくよく考えると、cumulative sumを使わなくても以下のようにすればよかった。というかこっちはspaceが定数ですむ。

```py
max_sum = -10000

for i in range(len(nums)):
    sum_nums = 0
    for j in range(i, len(nums)):
        sum_nums += nums[j]
        max_sum = max(max_sum, sum_nums)
return max_sum
```



### DP  = O(n) time

### 所感

560をやっていて、思い出して復習してみた。
cumulative sum(累積和)の復習をするつもりだったが、この問題は、よくよく考えると累積和を使ったらspaceが無駄になる。

多分cumulative sumが活躍するのは、クエリが複数回飛んでくる場合に備えて前処理をするケース。
それ以外のケース、特に左端と右端を総当たりするケースでは、計算時間がO(n**2)かかるので、上記のスニペットのように二重ループを回すほうがspaceはsaveできる。

two pointersでは解けないが、runnign sumだと解ける問題は TODO: リンク貼る　leetcode zero sum range

## 560. [Subarray Sum Equals K](https://leetcode.com/problems/subarray-sum-equals-k/)


### 解法１ 「cumulative sums => Counterのhashmap」O(n) 時間、O(1)スペース


- => と書いたが、これone pathで解ける。
- => と書いたが、one pathは結構時間掛かるので2pathでやるほうが安全かも。

todo: それぞれのケースをかく


### 解法２  cumulatve smsをとってからの二重ループ O(n**2)時間、O(n)スペース

解法３の下位互換。
### 解法３ ただの二重ループ O(n**2)、O(n)スペース


### 所感

配列に負の数が含まれることでかなり複雑化している。と思ったが、そうでもないのかも。
「Cumulative sumをとってそのあとCounterでhash mapを作成する」という定石に従えば瞬殺だった。

ちなみに、正の数のみのstrictly increasing orderの配列なら、以下のよう。(532の解法2では似たような2pointersの実装で、各if節の中でwhileループを回しているが、二重にしないほうが簡単に書ける気がする)


```python
        left, right = 0, 1
        if cumulative_sums[1] - cumulative_sums[0] == k:
            ans = 1
            
        while True:
            print(left, right)
            if (left == right) or (cumulative_sums[right] - cumulative_sums[left] <= k):
                right += 1
                if right == len(cumulative_sums):
                    return ans

                if (cumulative_sums[right] - cumulative_sums[left] == k):
                    ans += 1
            else:
                left += 1
                if (left != right) and (cumulative_sums[right] - cumulative_sums[left] == k):
                    ans += 1
        return ans
```

## 889. [Construct Binary Tree from Preorder and Postorder Traversal](https://leetcode.com/problems/construct-binary-tree-from-preorder-and-postorder-traversal/) (挫折)


### 所感

- https://leetcode.com/problems/construct-binary-tree-from-preorder-and-postorder-traversal/discuss/161268/C%2B%2BJavaPython-One-Pass-Real-O(N) を読もうとするも理解できず。

## 582. [Kill Process](https://leetcode.com/problems/kill-process/)


### 所感

簡単すぎて解説を読む気になれなかった。
pythonの内包表記は、`{ pid: i for i, pid in enumerate(pid) }`みたいな書き方が可能だということを学んだ。

## [947. Most Stones Removed with Same Row or Column](https://leetcode.com/problems/most-stones-removed-with-same-row-or-column/)


### 所感

UnionFindで、以下忘れてた。

- #union にて、最初に `self.are_in_the_same_group`ならその時点でreturnする
   - これを使わずに自前で実装するのなら、まずfindで親のidxを取得してそれらがイコール化を比較しないといけないのに、引数のxとyを比較していたw
- `union by rank`という名称


## 1048. [Longest String Chain](https://leetcode.com/problems/longest-string-chain/)



### 所感



## 

## 740. [Delete and Earn](https://leetcode.com/problems/delete-and-earn/)


### 所感

最適

## 255. [Verify Preorder Sequence in Binary Search Tree](https://leetcode.com/problems/verify-preorder-sequence-in-binary-search-tree/)


### 所感

binary searchを使う方向でトータル3時間くらい考えてたけど、これわんちゃん5分でできる問題だな。かなり良問。





## 284. [Peeking Iterator](https://leetcode.com/problems/peeking-iterator/)


Iteratorをwrapし、peekメソッドを持つ以下のようなiteratorクラスを実装せよ。

```
PeekingIterator(int[] nums) Initializes the object with the given integer array nums.
# int next() Returns the next element in the array and moves the pointer to the next element.
# bool hasNext() Returns true if there are still elements in the array.
# int peek() Returns the next element in the array without moving the pointer
```


### 解法

簡単。next_valというフィールドを持っておくだけ。初期化、nextの呼び出し後には必ずnext_valを必ず埋めておく(つまり、常に一つ先の要素を取り出しておく)、という実装をすることで、めっちゃシンプルに実装できるようになる。

```py
class PeekingIterator:
    def __init__(self, iterator):
        self._next = iterator.next()
        self._iterator = iterator

    def peek(self):
        return self._next

    def next(self):
        ret = self._next
        self._next = self._iterator.next() if self._iterator.hasNext() else None
        return ret
```


### 所感

`常に一つ先の要素を取り出しておく`という発想は思いつかなかった。脱帽。

## 341. [Flatten Nested List Iterator](https://leetcode.com/problems/flatten-nested-list-iterator/)

```
class NestedInteger:
#    def isInteger(self) -> bool:
#    def getInteger(self) -> int:
#    def getList(self) -> [NestedInteger]:   ##### 空配列を返すこともある！！！
```

上記のようなNestedIntegerの配列が渡される時、これをflattenして、順々に値を返すiteratorオブジェクトを実装せよ。実装するべきiteratorのinterfaceは以下。

```
class NestedIterator:
# NestedIterator(List<NestedInteger> nestedList) Initializes the iterator with the nested list nestedList.
# int next() Returns the next integer in the nested list.
# boolean hasNext() Returns true if there are still some integers in the nested list and false otherwise.
```

### 解法1. 初期化時にrecursionでtraverseしてlistを初期化 (思いついた)

まあ、これが一番に思いつく。ただ、iterator patternは、pointerだけを保持していいのでspaceをセーブできるというのが売りなので、そういう意味でこれを実装したら「僕デザインパターン知りません！」と堂々と言い放っているようなものなので注意。

あと、最初実装した時にわざわざrecursive funcからリストを返し、それを呼び出しがわでextendしていくという謎実装をしていた。そんなことしない、かくfunc内で(global変数あるいは)インスタンス変数のリストにappendしていく。


### 解法2. 初期化時にstackを使用してtraverseしてlistを初期化(思いついた)

解法1のrecursionの代わりにstackを用いただけ。最初にNestedIntegerの配列をリストに逆順につめ、これをstackとして用いる。
これも内部でまるっとデータ構造持っておかないといけないのでいい解法とは言えない。



### 解法3. stackを利用、#next()が呼ばれる度に次の要素までtraverse

解法2とアイデアは同じ。解法1のrecursionの代わりにstackを用いただけ。最初にNestedIntegerの配列をリストに逆順につめ、これをstackとして用いる。NestedIntegerはポインタでしかないので、spaceをsaveできる。ただし、与えられたリストの要素が全てintを表すNestedIntegerなら、解法2とspace complexityは同じという認識。


引っかかった点として、NestInteger.getList()は空配列になりうるというところ。したがって、`hasNext(), next()`が呼ばれた時点でtailの要素がlistのNestedIntegerだった場合、それを分解してやる(処理を繰り返す)必要がある。しかも、このロジックはhasNext()とnext()の両方で使用されるので、`def make_stack_top_an_integer(self)`としてメソッドに切り出すのが吉。

```
class NestedIterator:
    def __init__(self, nestedList: [NestedInteger]):
        self.stack = list(reversed(nestedList))
    
    def next(self) -> int:
        self.make_stack_top_an_integer()
        return self.stack.pop().getInteger()
    
    def hasNext(self) -> bool:
        self.make_stack_top_an_integer()
        return len(self.stack) > 0
```

### 解法4. Using a Generator

読み流した。yieldとかのお話。



### 所感

あとこの問題、「もう残りの要素がない状態で#next()が呼ばれたらどうしましょう？？？」って聞くと「こいつ優秀！」って思われるらしい。

解くこと自体は簡単。計算時間求めるのはややこい。
解答後、iterator patternについてまとめた。
https://github.com/kudojp/RoadToSuperCoder/blob/master/design_patterns.ipynb

振り返ってみるとかなり良問な気がしてきた。

## 117. [Populating Next Right Pointers in Each Node II](https://leetcode.com/problems/populating-next-right-pointers-in-each-node-ii/)

binary searchが与えられる。各nodeのnextフィールドに、その右隣のnodeを指すpointerを代入せよ。O(1)spaceで。


### 解法1 O(N) space

まあ、普通にqに同階層のnodeを詰めるBFSで実装する。


### 解法2 待望のO(1) space

BFS。現在traverseしているより一段下のlevelのnodesのnextフィールドを設定することで、qを使用しなくてもその階層の次のnodeをたどっていける。(一段下というのが肝)

```py

## 最初にゴニョゴニョ

while leftmost is not None:
    curr = leftmost
    prev = None
    leftmost = None
    while curr is not None:
        for child in [curr.left, curr.right]:
            if child is None:
                continue
            if prev is None:    # when it is a first node in the level
                leftmost = child
                prev = child
                continue
            prev.next = child
            prev = prev.next
        curr = curr.next
```


### 所感

本当にびっくりするくらい美しい良問だと思う。脱帽。




## 532. [K-diff Pairs in an Array](https://leetcode.com/problems/k-diff-pairs-in-an-array/)

int[]が与えられる時、差がkとなるような数字のユニークな組み合わせは何通りあるか？

### O(N)時間: counterのhashを作成後にkeyをループする

```py
val_to_cnt = Counters(nums)
checked = set()
cnt = 0

for val in val_to_cnt:
    if val in checked:
        continue
    if val + k in val_cnt:  # +kの場合のみ考えることでsimplifyした。
        cnt += 1
return cnt
```

上では書いてないが、k = 0の場合が見事なエッジケースw。

### O(NlogN)時間: ソートしてから2pointers

実装はややこい。書き終わってみたらまあなるほどって感じ。
気が向いたらかく　TODO

### 所感

解法２は、spaceは解法１と同じO(N)、timeは解法１より遅い上に、解法１より実装がややこしいので、まあ、解法１を使うでしょ。



[参考] ソートのtimecomplexityは `O(logN)` ~ `O(N)`かかる。
(meege sortがO(N), quick sortがO(logN)~O(N))

- https://twitter.com/kdjp20/status/1389091424063213568
- https://twitter.com/kdjp20/status/1389817538062680066


## 1648. [Sell Diminishing-Valued Colored Balls](https://leetcode.com/problems/sell-diminishing-valued-colored-balls/) (挫折)

## 1539. [Kth Missing Positive Number](https://leetcode.com/problems/kth-missing-positive-number/)


strictly increasing orderのintのlistが与えられる。skipされた数字のうちk番目のものを求めよ。(e.g. [1,2,4,6], k=2 -> 5)

### 解法1 (できた)

前からループを回す。計算時間O(N)

### 解法２ (できた)

binary searchを使う。
配列のあるindexまでにどれだけ数字がskipされたかは、arr[idx] - idx で求められることを利用。


bisect_left的実装でidxを見つけた後、そのidxからbackwardに戻っていくのではなく、idx - 1からforwardに探索してやるとより直感的。

### 所感



いい binary searchの練習になった。解法2で、一つidxを戻してforwardにたどっていくのわかりやすいなーって思った。

## 875. [Koko Eating Bananas](https://leetcode.com/problems/koko-eating-bananas/)

複数のバナナの山(`int[]`)をh時間内に食べ切ることのできる最も遅いスピードを求めよ。ただし、同じ時間フレーム(1時間)以内に複数のバナナに手をつけることはできない。

### 解法1 (heapを使用、思いついた -> tle) 

計算時間がO(h logN)かかってしまいTLE。    
TODO: リンク貼る

### 解法2 (binary searchを使用し、O(N * log max_pile_size)

計算時間を`O(N * log max_pile_size)`まで縮められた。   
(bisect_leftの実装にミスって116/118でTLEを連発してしまった。Javaで書き直したが、そういう問題ではなかったw)    


```python    
low = math.ceil(sum(piles) / h)
high = max(piles)

# イメージは、[f,f,f,f,t,t,t,t,t](x)の中で、一番左のtのindexを取得する。
# これが一番の肝。helper functionでtaking_timeなんかを取ってhと比較したりするとややこしい。

while low < high:
    mid = (low + high) // 2
    is_possible = is_possible(mid)
    
    if is_possible: # speed must be slower than mid
        high = mid
    else: # h <= t
        low = mid + 1 # speed is higher or 
    return low     # low != len(piles)
```
        
        
### 所感

解法2はグッチさんとマックで1時間くらい考えてやっと思いついた。
helper_functionをうまく設定できるかが鍵だと思う。    
bisect_leftに関してまとめた。 https://twitter.com/kdjp20/status/1424250217268871170

## 300.  [Longest Increasing Subsequence](https://leetcode.com/problems/longest-increasing-subsequence/)

intの配列において、最長のLISの配列の長さを求める問題。O(NlogN)時間でとけ。
(ex) [0,3,1,6,2,2,7] → [0,1,2,7] なので、ans = 4

### 解法1 (似たようなの思いついた)

与えられた配列の各要素に対応し、その要素までのLISの配列長を表すリストdpを作成していく。
初期条件は、dp[0] = 1。
dp[1:]に関しては、max(それまでに登場した要素で、現在の要素の値未満の要素のLIS長)+1。なお、そこまでにその値未満の要素が存在しない場合には1を入れる。

### 解法2 (全く思いつかなかった)

dpリストを作成し、これをupdateしていく。初期値は、dp=[nums[0]]。与えられた配列nums[1:]の要素を順々に見ていき、その要素がdpのどの要素よりも大きい場合には、それをdpの末尾にappendする。そうでない場合には、dpの中でそれ以上の値でindexが最も若い要素とそれを入れ替える。最終的に完成したlen(dp)が、求める値である。


```
着眼点としては、[1, 5, 3]が与えられた時、「３が加わる瞬間に５は用無し」になること。
注意点としては、「最終的に得られるdpリストの長さは、LSIの最大長に等しい」が、「それらの要素がLSIを作る要素を表しているわけではない」ということ。例えば、[1,3,5,2]が与えられた時、dpは、[1]>[1,3]>[1,3,5]>[2,3,5]となるが実際にLSIを作るのは[1,3,5]。
```

なお、「dpの中でそれ以上の値の一つ目の要素」を発見する過程でbinary searchを用いることでO(NlogN)時間になる。



### 所感

解法２が天才的。どうやったら思いつくんだ、、、
bisect_leftを使ったが、特に場合分けが要らなかった。これは、bisect_left(list, x)がlen(list)を返すケースを考えなくて良いから。


## 75. [Sort Colors](https://leetcode.com/problems/sort-colors/)

`Dutch national flag problem`   
0,1,2のいずれかの要素からなる配列(e.g. [1,2,2,1,0])を0, 1, 2のセクションに分かれるように、inplaceに入れ替えろ。

### 解法1 (my solution)

- １巡目に、0,1,2のそれぞれの出現数cnt0, cnt1, cnt2を数える。
- ２巡目に、(left, right) = (0, cnt1)から初めて、先頭のcnt0個の要素が0になるように(1or2 <-> 0)のswapを繰り返す。
- ３巡目に、(left, right) = (cnt0, cnt0+cnt1)から初めて、真ん中のcnt1個の要素が1になるように(1 <-> 2)のswapを繰り返す。


### 解法2 (似たようなの思いついたけど、edge caseをがややこそうだったので撤退)

次に、0, 2を置かれるべきindexをそれぞれ(p0, p1) = (0, len(nums)-1)に、またcurr = 0を定義する。


```python
while curr <= p1:   # =が必要らしい
    if curr == 0:
        (p0とcurrをswap)
        p0 += 1
        curr += 1       # currも右に動かすらしい。これは、currがp0よりも後ろにいるのを防ぐため??
    elif curr == 1:
        curr += 1
        # p1とのswap後には、currの箇所に２が来るかもしれないからまだcurrを右に動かすことはできない
    else:
        (p1とcurrをswap)
```

### 所感

解法１ではエッジケースを考え漏れていた。(cnt0=len(nums)つまり0のみの配列の場合や、cnt0+cnt1=len(nums)つまり0と1のみの配列の場合)
解法２は意外にもエッジケースを考える必要はなかった。ただし、コメントに書いた箇所の考慮がまたしっくりきていない。
